In [47]:
import tensorflow as tf
import pandas as pd
import gpt_2_simple as gpt2
from datetime import datetime
from transformers import GPT2Tokenizer,GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch

In [3]:
scripts = pd.read_csv('scripts.csv')
scripts['line'] = scripts.Character + ': ' + scripts.Dialogue
#scripts.line[-1:] = scripts.line[-1:]  + ' <|endoftext|>'

In [4]:
class seinfeld_lines(Dataset):  
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lines = []

        for row in scripts['line']:
          self.lines.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row}<|endoftext|>")
            ))               
        if truncate:
            self.lines = self.lines[:20000]
        self.lines_count = len(self.lines)
        
    def __len__(self):
        return self.lines_count

    def __getitem__(self, item):
        return self.lines[item]
    
dataset = seinfeld_lines(scripts['line'], truncate=True, gpt2_type="gpt2")

In [5]:
scripts.line = scripts.line.astype(str)

In [44]:
run_name = 'fine_tuning_run_1'
model_size = '124M'

In [45]:
scripts[-1:]

,Unnamed: 0,Character,Dialogue,EpisodeNo,SEID,Season,line
54615,54615,JERRY,"Alright, hey, you've been great! See you in th...",23.0,S09E23,9.0,"JERRY: Alright, hey, you've been great! See yo..."


In [16]:
#Get the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2',padding_side='left')
model = GPT2LMHeadModel.from_pretrained('gpt2')

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [17]:

def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=1, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [18]:
model = train(dataset, model, tokenizer)

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training epoch 0
0


20000it [22:18, 14.94it/s]


In [66]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=1,
    entry_length=100, #maximum number of words
    top_p=0.9,
    temperature=.75,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
                
    return generated_list

#Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data):
  generated_lines = []
  for i in range(len(test_data)):
    x = generate(model.to('cpu'), tokenizer, test_data['Character'][i], entry_count=1)
    generated_lines.append(x)
  return generated_lines

In [23]:
#Run the functions to generate the lines
jerry = scripts[scripts.Character=='JERRY'].reset_index()
george = scripts[scripts.Character=='GEORGE'].reset_index()
elaine = scripts[scripts.Character=='ELAINE'].reset_index()
kramer = scripts[scripts.Character=='KRAMER'].reset_index(),
frank = scripts[scripts.Character=='FRANK'].reset_index()
newman = scripts[scripts.Character=='NEWMAN'].reset_index()
estelle = scripts[scripts.Character=='ESTELLE'].reset_index()

In [67]:
george_elaine=[]
george_elaine.append(generate(model,tokenizer,prompt='Elaine: (to George) have you tried the new dating app that matches you with other people who'))
george_elaine.append(generate(model, tokenizer, prompt="George: it didnt work because"))
print(george_elaine)

100%|██████████| 1/1 [00:12<00:00, 12.88s/it]

[['Elaine: (to George) have you tried the new dating app that matches you with other people who are interested in you? George: (to her) uh uh uh. I\'ve been looking for a girl with some sort of artistic flair who is willing to talk to you, but I don\'t know. I\'m just a big guy who likes to chat, and I want to make sure that I\'m getting the same kind of response that you get. (pause) This is so odd, George. You\'ve seen a lot of young men come up to me and say, "Hey,<|endoftext|>'], ["George: it didnt work because of a bug in the controller...the game couldn't do anything about it, so it was basically just a cheap way to have a tutorial on how to play.\n\n\nmatthew: i had to change the whole setup because the controller was just too much. the only thing i had to change was the controller itself, so it made me feel less alone. i had to do a lot of crazy things to make this a game that was so fun to play.\n\n\nvlad:<|endoftext|>"]]


In [41]:
# Import the huggingface library
import huggingface

# Load the GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Preprocess your text data
processed_data = scripts['line']

# Split the data into training and validation sets
#train_data, val_data = train_test_split(processed_data, test_size=0.2)

# Choose a model architecture and training parameters
model_params = {
    'batch_size': 64,
    'learning_rate': 1e-5,
    'num_epochs': 1
}

# Train the model on the training data
model.train(processed_data, model_params , tokenizer)

# Evaluate the model's performance on the validation set
#val_loss = model.evaluate(val_data)
#print(f'Validation Loss: {val_loss:.4f}')

# Generate an entire script for an episode of Seinfeld
generated_script = model.generate("Generate an entire script for an episode of Seinfeld, set in a coffee shop, featuring Jerry, Elaine, and George.")
print(generated_script)


TypeError: train() takes from 1 to 2 positional arguments but 4 were given

In [19]:
model.generate()

In [27]:
text_generation(estelle)

100%|██████████| 1/1 [00:02<00:00,  2.57s/it]


[['ESTELLE VS. HIGH-TECH (DESIGNER: ROBERT CRUZELMAN)\n\nVALERIE VS. MATT SCAL<|endoftext|>'],
 ['ESTELLE, who has been in "management" as by appointment, from now on is an "owner/operator," to whom she has no say. She<|endoftext|>'],
 ["ESTELLE: All right, let's go.\n\nJUROR: Yes, please. I want you to watch it.\n\nSAVAGE<|endoftext|>"],
 ['ESTELLE: You listen to her do?"\n\nMOYER: Yes.\n\n[giggles]\n\nKRAMER: Yes,<|endoftext|>'],
 ['ESTELLE: Tonight on Freak Show with Jay Leno, we talk about a new album called NEONER AND ASIAN WOMEN, about co-author<|endoftext|>'],
 ["ESTELLE, HILLARY (R), BURKE/JERRY: You do what you do best. Nobody's perfect.<|endoftext|>"],
 ['ESTELLE: I do know that this is very funny, when I heard this, I said, of course, if they give you money, you get nothing<|endoftext|>'],
 ['ESTELLE, Mich. — Nick Griffin can imagine himself at home in the shiny new Volkswagen Polo.<|endoftext|>'],
 ['ESTELLE: What do you think? Is this with us?\n\nSTEWART: Good. First, I wa